In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

### Read this First

#### Remember that `tab` is is useful for autocompletion.

#### Remember that `shift + tab` is useful for rapidly obtaining usage + documentation.

###**Optical Flow**

**Throughout the class, we've been looking at many ways to analyze and extract spatial information from images. Now we'll briefly talk about how we can use the temporal information inherent in series of images to learn more about the content of the images.**

**One way of getting information temporally is through optical flow, which is the apparent motion of brightness patterns in the image. Ideally, optical flow would be the same as the motion field, which is a representation of real world 3D motion. So by finding the optical flow field, we can hopefully observe real world motion behaviour of objects that are present in the image.**

**Let's first import some things.**

In [ ]:
from time import sleep

import numpy as np
from scipy import signal
from scipy.ndimage.filters import gaussian_filter

import cv2
import matplotlib.pyplot as plt
from IPython.display import clear_output
from google.colab.patches import cv2_imshow

**Read in the gif that is part of the repo, of three cars moving. We'll display it as a series of imshows with a small delay in between - you might see some flashing as the images change from one frame to another (I couldn't really figure out a nice way to display gifs in google colab).**

In [ ]:
gif = cv2.VideoCapture('taxi.gif')

for i in range(int(gif.get(cv2.CAP_PROP_FRAME_COUNT))):
  ret, frame = gif.read()
  frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  clear_output(wait=True)
  cv2_imshow(frame_gray)
  sleep(0.4)

**The object produced by the VideoCapture function isn't really nice to deal with, as we can't index through it nicely, and we want to work with grayscale images as well. Hence, below, find the number of frames, height, and width of the image sequence.**

**Then create a 3D matrix called "frames" of size num_frames $\times$ height $\times$ width, that stores the grayscaled frames of the original gif. To check that this works, display the first frame using plt.imshow. (Don't worry if you see that it doesn't start at (0, 0) for the plot)**

In [ ]:
gif = cv2.VideoCapture('taxi.gif')

num_frames = ?
height = ?
width = ?
frames = ?

**In lecture, we saw that to get the optical flow, we assumed brightness contancy, small motion, and spatial coherence. This allows us to use gradients within a frame and between frames to determine motion of pixels from one frame to another.**

**Fill out the function below that takes two frames, im1 and im2, and computes the $\sum I_xI_x$, $\sum I_xI_y$, and $\sum I_yI_y$ terms for the second frame, and $\sum I_xI_t$ and $\sum I_yI_t$ terms between the two frames.**

**For $I_x$ and $I_y$ use np.gradient of the second frame, and $I_t$ should be the pixel-wise difference between the two frames. $I_xIx$, $I_yI_y$, $I_xI_y$, $I_xI_t$, and $I_yI_t$ are computed with pixel wise multiplication as usual.**

**To compute the $\sum$, at every pixel sum up the relevant terms in a gaussian window around that pixel, using scipy.ndimage.gaussian_filter with sigma=3 (This is like what we did with the Harris Corner Detector in Week 4 notebook.**

In [ ]:
def getGradients(im1, im2):
  """ Computes the gradients that are needed to find optical flow between im2 and im1.
      Args:
        im1: First frame of size [height, width]
        im2: Second frame of size [height, width]
      Returns:
        i_xx_sum: 2D matrix of size [height, width] that contains sum of I_xI_x values in a gaussian window at each pixel
        i_xy_sum: 2D matrix of size [height, width] that contains sum of I_xI_y values in a gaussian window at each pixel
        i_yy_sum: 2D matrix of size [height, width] that contains sum of I_yI_y values in a gaussian window at each pixel
        i_xt_sum: 2D matrix of size [height, width] that contains sum of I_xI_t values in a gaussian window at each pixel
        i_yt_sum: 2D matrix of size [height, width] that contains sum of I_yI_t values in a gaussian window at each pixel
  """

  return i_xx_sum, i_xy_sum, i_yy_sum, i_xt_sum, i_yt_sum

**Why do we sum the gradient terms in a window around each pixel? What assumption are we making here that allows us to do this?**

**Using those gradients, we can create a least squares equation that is similar to the Harris Corner Detector. In the function below, fill out the missing lines to finish the function to find optical flow.**

**Note that when we solve for $[u, v]^T$, we don't use np.linalg.lstsqr or some other method - we can solve for it directly, as from slide 39 we see $d=(A^TA)^{-1}(-A^Tb)$, and we check beforehand whether $A^TA$ is invertible.**

In [ ]:
def optical_flow(im1, im2, stride):
  """ Computes the optical flow between frames im2 and im1.
      Args:
        im1: First frame of size [height, width]
        im2: Second frame of size [height, width]
        stride: Determines how dense you want the optical flow field to be
      Returns:
        u: x value of optical flow vector of every pixel
        v: y value of optical flow vector of every pixel
  """

  # Calls our previous function to get the image gradients
  height, width = im1.shape
  i_xx_sum, i_xy_sum, i_yy_sum, i_xt_sum, i_yt_sum = getGradients(im1, im2)

  # Create containers for storing the u, v components of optical flow vector for each pixel
  u = np.zeros(im1.shape)
  v = np.zeros(im2.shape)

  # Loop through every stride pixels
  for i in range(0, height, stride):
    for j in range(0, width, stride):

      # Find the ATA matrix for this pixel (see slide 39)
      ## TODO

      # Find the ATb matrix for this pixel (see slide 39)
      ## TODO

      # Compute the determinant of the ATA matrix, and if it zero, we move on to the next pixel
      ## TODO

      # Directly solve for the u, v terms
      ## TODO
      u[i, j] = ?
      v[i, j] = ?

  return u, v

**We can now test it out by running our gif that we have stored in frames below. Call optical_flow(frames[i-1], frames[i], 1) for each of our frames, and store the resulting u, v fields in u_list and v_list. If it's taking too long, you can change the stride to be 2 instead of 1. (It takes around 90 seconds for stride=1 and around 23 seconds for stride=2 for me)**

In [ ]:
u_list = np.zeros((num_frames, height, width))
v_list = np.zeros((num_frames, height, width))

**Let's visualize the optical flow field gradient. Below, for every frame, compute the optical flow gradient $u^2+v^2$ at every pixel and use plt.imshow with cmap='hot' to display this.** 

In [ ]:
for i in range(num_frames):
  clear_output(wait=True)

  # Display the image here
  ## TODO
  
  plt.show()
  sleep(0.3)

**You should be seeing three car-shaped areas that move just like how the cars move in the original gif.**

**Finally, we can visualize the optical vector field directly as well, using plt.quiver.**

**Similar to what was done above, plot and display the optical vector field for each frame in a sequence. Sample the pixels so that you only display the optical field vector for every 10th pixel (doesn't have to be exactly 10; just make sure that it's not too crowded or too sparse).**

**Since plt.quiver is a plot, you might want to invert the y axis using plt.gca().inverse_yaxis() so it's an image.**